In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import requests
from bs4 import BeautifulSoup
import lxml
import json
from urllib.parse import quote

# from futsim_funcs import TPR, TFR, TPSR, TGKR, TDR

In [2]:
import warnings
warnings.filterwarnings("ignore", message="The default value of numeric_only in DataFrame.mean is deprecated.", category=FutureWarning)

In [3]:
# players_df = pd.read_csv("players_db/fm23/platinum22_fm23data.csv")
players_df = pd.read_csv("../fifa24_db/pdb_23.csv")
# players_df = pd.read_csv("players_db/fm23/fm23db_processed.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../fifa24_db/pdb_23.csv'

In [ ]:
tpr_data = pd.read_csv("players_db/fm23/team_ratings.csv")

In [ ]:
tpr_data.sort_values("tpr", ascending=False).head(n=5)

In [ ]:
leagues = ["Premier League", "Serie A", "Ligue 1", "LaLiga", "Bundesliga"]
fbref_league_dict = {
    "Premier League": {"name": "Premier League", "fbref_league_id": 9, "league_id": 13},
    "Serie A": {"name": "Serie A", "fbref_league_id": 11, "league_id": 31},
    "Ligue 1": {"name": "Ligue 1", "fbref_league_id": 13, "league_id": 16},
    "LaLiga": {"name": "LaLiga", "fbref_league_id": 12, "league_id": 53},
    "Bundesliga": {"name": "Bundesliga", "fbref_league_id": 20, "league_id": 19},   
}

<br><br><br><br>

# FBRef Match Logs URLs

In [ ]:
def GetTeamCredentials(league_id, season="2022-2023"):
    league_url = f"https://fbref.com/en/comps/{league_id}/{season}"
    response = requests.get(league_url)
    soup = BeautifulSoup(response.content, "lxml")

    table = soup.find("table", {"class": "stats_table"})
    team_ids = []
    team_names = []

    for row in table.find_all("tr")[1:]:
        team_name = row.find("td", {"data-stat": "team"}).text
        team_id = row.find("td", {"data-stat": "team"}).a.get("href").split("/")[3]
        team_ids.append(team_id)
        team_names.append(team_name)
    return {
        "team_ids":team_ids,
        "team_names":team_names,
    }

In [ ]:
from rapidfuzz import process
def find_best_match(name, choices):
    return process.extractOne(name, choices)

In [ ]:
def TPRFromId(tpr_data, club_team_id):
    power = tpr_data.query(f"club_team_id == {club_team_id}")["power"].iloc[0]
    return {
        "power": power,
    }

In [ ]:
def TFRFromId(tfr_data, club_team_id):
    tfr_data = tfr_data["data"]
    power = tfr_data.query(f"club_team_id == {club_team_id}")["power"].iloc[0]
    finishing = tfr_data.query(f"club_team_id == {club_team_id}")["finishing"].iloc[0]
    return {
        "power": power,
        "finishing": finishing,
    }

In [ ]:
def TeamNameIdConverter(tpr_data, team_name, league_id, print_output=False):
    # TeamNameIdConverter(tpr_data,"chelseas",13,1)
    club_team_names = tpr_data.query(f"league_id == {league_id}")["club_name"].tolist()
    club_team_ids = tpr_data.query(f"league_id == {league_id}")["club_team_id"].tolist()
    club_team_name, score, other = find_best_match(team_name, club_team_names)
    club_team_id = club_team_ids[club_team_names.index(club_team_name)]
    if print_output:
        print(f"ID for {club_team_name}:")
    return club_team_id

In [ ]:
def FB_URLS(creds, tpr_data, stat_attribute, fbref_league_id, league_id, season="2022-2023"):
    tpr_data = tpr_data
    team_ids = creds["team_ids"]
    team_names = creds["team_names"]
    club_team_names = tpr_data.query(f"league_id == {league_id}")["club_name"].tolist()
    club_team_ids = tpr_data.query(f"league_id == {league_id}")["club_team_id"].tolist()

    for team_id, team_name in zip(team_ids, team_names):
        best_match = find_best_match(team_name, club_team_names)
        if best_match:
            club_team_name, score, other = best_match
            club_team_id = club_team_ids[club_team_names.index(club_team_name)]
            match_logs_url = f"https://fbref.com/en/squads/{team_id}/{season}/matchlogs/c{fbref_league_id}/{stat_attribute}/{team_name.replace(' ', '-')}"
            match_logs_stats_dict[stat_attribute].append({
                "id": team_id,
                "league_id": league_id,
                "fbref_league_id": fbref_league_id,
                "stat": stat_attribute,
                "team": team_name,
                "club_team_id": club_team_id,
                "club_team_name": club_team_name, 
                "url": match_logs_url
            })
        else:
            print(f"No match found for team: {team_name}")

<br><br><br><br>
<h1 style="color:green">Scraping FBRef</h1>

In [ ]:
df = pd.read_html("https://fbref.com/en/squads/b8fd03ef/2022-2023/matchlogs/c9/shooting/-Manchester-City")[1][:38]
df.sort_values(('Against Manchester City', 'Date')).head()
df.columns

In [ ]:
def test_stat_df(stat_attribute):
    df = pd.read_html(f"https://fbref.com/en/squads/b8fd03ef/2022-2023/matchlogs/c9/{stat_attribute}/-Manchester-City")[0][:38]
    url = f"https://fbref.com/en/squads/b8fd03ef/2022-2023/matchlogs/c9/{stat_attribute}/-Manchester-City"
    return url

In [ ]:
def FB_MatchLogs(url, last_matchweek=38):
    encoded_url = quote(url, safe=':/')
    df=pd.read_html(encoded_url)
    return {
        "for": df[0][:last_matchweek],
        "against": df[1][:last_matchweek],
    }

In [ ]:
def MatchLogs_MultiIndexColumnDict(df, stat_attribute=None):
    stat_columns_dict = {}
    for item in df.columns:
        right_side = item[1]
        stat_columns_dict[right_side] = item
    if stat_attribute:
        return stat_columns_dict[stat_attribute]
    return stat_columns_dict

<br><br><br><br>
<h1 style="color:aqua">Regression Boilerplate</h1>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def train_linear_regression(df, features, target_variable, apply_dtype, test_size=0.19, random_state=42):
    X = df[features]
    y = df[target_variable]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    df['predicted'] = model.predict(df[features])
    df['predicted'] = df['predicted'].apply(apply_dtype)
    
    mse = mean_squared_error(y_test, y_pred)
    coefficients = dict(zip(features, model.coef_))
    intercept = model.intercept_
    return model, mse, coefficients, intercept

def model_error(df, target_variable):
    df['error'] = df[target_variable] - df['predicted']
    return df['error'].apply(abs)

def model_head(df, features, target_variable, sort_parameter="team_power", sort_ascending=False, n=5):
    return df[features+[target_variable,"predicted","error"]].sort_values(sort_parameter, ascending=sort_ascending).head(n=n)

def model_equation(coefficients, intercept):
    equation = " + ".join([f"{coef:.2e} * {feat}" for feat, coef in coefficients.items()])
    equation = equation + f" + {intercept:.2e}"
    equation = equation.replace("+ -", "- ")
    return equation

def model_predict(**args):
    pass

def plot_predicted_vs_actual(df, target_variable, model=None, features=None):
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df[target_variable], marker='o', linestyle='-', color="b", label='Actual')
    
    if model and features:
        plt.plot(df.index, df['predicted'], marker='o', linestyle='-', color="r", label='Predicted')
    plt.title('Line Plot of Predicted Values')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def convert_equation_to_json(features, values, intercept):
    # Usage: convert_equation_to_json(features, coefficients.values(), intercept)
    equation_dict = {}
    for feature, value in zip(features, values):
        equation_dict[feature] = {
            "name": feature,
            "value": value
        }
    equation_dict["intercept"] = {"name":"intercept", "value":intercept}
    return equation_dict

<br>
<h1 style="color:aqua">Randomness Factor</h1>

In [ ]:
def randomness_proportional(df, target_variable):
    target_mean = df[target_variable].mean()
    target_std = df[target_variable].std()
    target_min = df[target_variable].min()
    target_max = df[target_variable].max()
    formula = (target_min) / (target_mean) 
    print("Min:",target_min, ",", "Max:",target_max)
    return formula

def randomness_volume(df, target_variable):
    target_mean = df[target_variable].mean()
    target_std = df[target_variable].std()
    target_min = df[target_variable].min()
    target_max = df[target_variable].max()
    formula = (target_mean) / (target_std)
    print("Min:",target_min, ",", "Max:",target_max)
    return formula

In [ ]:
def interval_proportional(value, step=0.05, expand_multiplier=1):
    lower_bound = round(value - step, 3)
    upper_bound = round(value + step, 3)
    numbers = np.arange(-upper_bound*expand_multiplier, upper_bound + step*expand_multiplier, step)
    return numbers

def interval_volume(value, step=1, expand_multiplier=1):
    lower_bound = int(value - step)
    upper_bound = int(value + step)
    numbers = np.arange(-upper_bound*expand_multiplier, upper_bound + step*expand_multiplier, step)
    return numbers

In [ ]:
def probability_protector(value):
    if value < 0:
        return 0
    elif value > 1:
        return 1
    else:
        return value
    
def negativity_protector(value, raw_value):
    if value < 0:
        return raw_value
    else:
        return value

In [ ]:
def random_choice(array):
    return np.random.choice(array)

<br><br><br><br>
<h1 style="color:pink">Match Logs DataFrame</h1>

In [ ]:
def ColumnConverter(df, keyword):
    df_cols = df.columns.map('.'.join)
    new_cols = [col.split('.')[1] if keyword in col else col for col in df_cols]
    return new_cols

In [ ]:
def prepare_expand_dict(column_dict, x, y):
    # example: {"unnamed.0.st: "}
    keys = column_dict.keys()
    values = column_dict.values()
    expanded_dict = {}
    for key, value in zip(keys, values):
        expanded_dict["team_"+key.lower()] = x[value]
        expanded_dict["opponent_"+key.lower()] = y[value]
    return expanded_dict

In [ ]:
def prepare_rating_dict(ratings_array, team_id, suffix):
    dict_ = {}
    for rating in ratings_array:
        metric_data = rating["data"].query(f"club_team_id == {team_id}")[rating["name"]].iloc[0]
        dict_[suffix+"_"+rating["name"]] = metric_data
    return dict_

In [ ]:
def generate_fbref_match_logs_array \
(match_logs_stats_dict, column_dict, tpr_data, \
 other_rating_data, league_id, team_number_league, last_matchweek, current_stat, multiple_array=[]):

    df_log_array = multiple_array
    tpr_data = tpr_data["data"]
    
    for tindex in range(team_number_league):
        time.sleep(0.01)
        current_team = match_logs_stats_dict[current_stat][tindex]

        url = current_team["url"]
        print(url, tindex)

        match_logs = FB_MatchLogs(url, last_matchweek=last_matchweek)
        df0 = match_logs["for"]
        df0.columns = ColumnConverter(df0, "For")
        
        df1 = match_logs["against"]
        df1.columns = ColumnConverter(df1, "Against")
        
        team_names = df0["Opponent"].unique().tolist()
        team_ids = [TeamNameIdConverter(tpr_data, team_name, league_id) for team_name in team_names]
        team_rating_data = TFRFromId(tfr_data, current_team["club_team_id"])

        for i in range(last_matchweek):
            x = df0.iloc[i]
            y = df1.iloc[i]
            x_rating_data = prepare_rating_dict(other_rating_data, current_team["club_team_id"], suffix="team")
            y_rating_data = prepare_rating_dict(other_rating_data, TeamNameIdConverter(tpr_data, x["Opponent"], league_id), suffix="opponent")
            lab_df_dict = {
                "league_id": league_id,
                "team_name": current_team["club_team_name"],
                "team_id": current_team["club_team_id"],
                "opponent_name": x["Opponent"],
                "opponent_id": TeamNameIdConverter(tpr_data, x["Opponent"], league_id),
                "team_power": x_rating_data["team_power"],
                "opponent_power": y_rating_data["opponent_power"],
            }
            merged_dict = {**lab_df_dict, **prepare_expand_dict(column_dict, x, y), **x_rating_data, **y_rating_data}
            df_log_array.append(merged_dict)
    print("Data collecting has been completed! ✅")
    return df_log_array

<br><br><br><br>
<h1 style="color:orange">🦸‍♂️Must Parameters</h1>

In [ ]:
match_logs_stats_dict = {
    "shooting":[],
    "passing":[],
    "defense":[],
    "keeper":[],
    "passing_types":[],
    "gca":[],
    "possession":[],
    "misc":[],
}

attributes = match_logs_stats_dict.keys()
league_name = "Serie A"
league_id = fbref_league_dict[league_name]["league_id"]
fbref_league_id = fbref_league_dict[league_name]["fbref_league_id"]
season = "2022-2023"
team_number_league = 20
last_matchweek = 38
creds = GetTeamCredentials(fbref_league_id, season)

for stat_attribute in attributes:
    FB_URLS(creds, tpr_data, stat_attribute, fbref_league_id, league_id)

In [ ]:
current_stat = "shooting"

In [ ]:
proper_terms = ['Date', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
                'Opponent', 'Goals', 'Shots', 'Shots on Target',
                'Shots on Target Percentage', 'Goals per Shot', 'Goals per Shot on Target',
                'Distance Covered', 'Free Kicks', 'Penalties', 'Penalty Attempts',
                'Expected Goals', 'Non-Penalty Expected Goals', 'Expected Goals per Shot',
                'Goals minus Expected Goals', 'Non-Penalty Goals minus Expected Goals',
                'Match Report']
column_names = ['Date', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
                'Opponent', 'Standard.Gls', 'Standard.Sh', 'Standard.SoT',
                'Standard.SoT%', 'Standard.G/Sh', 'Standard.G/SoT', 'Standard.Dist',
                'Standard.FK', 'Standard.PK', 'Standard.PKatt', 'Expected.xG',
                'Expected.npxG', 'Expected.npxG/Sh', 'Expected.G-xG',
                'Expected.np:G-xG', 'Unnamed: 24_level_0.Match Report']

proper_terms = [term.lower().replace(" ", "_") for term in proper_terms]

column_dict = dict(zip(proper_terms, column_names))

In [ ]:
prepared_index_dict = column_dict
prepared_rating_data = [tpr_data, tfr_data]

In [ ]:
# df_log_array = generate_fbref_match_logs_array \
# (match_logs_stats_dict, prepared_index_dict, tpr_data,  \
#  prepared_rating_data, league_id, team_number_league, last_matchweek, current_stat)
# df_log = pd.DataFrame(df_log_array)
# df_log.to_csv(f"../@blacksmith/match_logs/fbref_match_logs_{league_name}@{current_stat}.csv")

df_log = pd.read_csv(f"../@blacksmith/match_logs/fbref_match_logs_{league_name}@{current_stat}.csv")

<br><br><br><br>
<h1 style="color:yellow">"shooting"</h1>

<br><br><br><br>
<h1 style="color:red">Total Shot Volume</h1>

In [ ]:
df_log.columns

In [ ]:
columns_to_drop = ['team_name','team_id','opponent_name','opponent_id']
df = df_log
df["total_shots"] = df["team_shots"] + df["opponent_shots"]

In [ ]:
features = ['team_power', 'team_finishing', 'opponent_power', 'opponent_finishing', "team_shots", "opponent_shots"]
target_variable = 'total_shots'

In [ ]:
model, mse, coefficients, intercept = train_linear_regression(df, features, target_variable, int)
mean_error = model_error(df, target_variable).mean()
std_error = model_error(df, target_variable).std()
print("Mean Squared Error:", mse)
print("Error Mean:", mean_error)

In [ ]:
# plot_predicted_vs_actual(df, target_variable, model=model, features=features)

In [ ]:
model_equation(coefficients, intercept)

In [ ]:
model_head(df, features+columns_to_drop, target_variable, sort_parameter="predicted", sort_ascending=True, n=5)

In [ ]:
randomness_func, interval_func = randomness_volume, interval_volume
result = randomness_func(df, target_variable)
interval = interval_func(result, step=1)

print("Result:", result)
print("Interval:", interval)
random_choice(interval)

In [ ]:
df["randomness"] = df["predicted"].copy()
df["randomness"] = df["predicted"].apply(lambda value: negativity_protector(value + random_choice(interval), value))

df[features+columns_to_drop+[target_variable,"predicted","randomness"]].sort_values("randomness", ascending=False).head(n=5)

<br><br><br><br>
<h1 style="color:red">Shot Share</h1>

In [ ]:
df["team_sh_share"] = df["team_shots"] / df["total_shots"]

In [ ]:
features = ['team_power', 'opponent_power']
target_variable = 'team_sh_share'

In [ ]:
model, mse, coefficients, intercept = train_linear_regression(df, features, target_variable, float)
mean_error = model_error(df, target_variable).mean()
std_error = model_error(df, target_variable).std()
print("Mean Squared Error:", mse)
print("Error Mean:", mean_error)

In [ ]:
model_equation(coefficients, intercept)

In [ ]:
# plot_predicted_vs_actual(df, target_variable, model=model, features=features)

In [ ]:
model_head(df, features+columns_to_drop, target_variable, sort_parameter="predicted", sort_ascending=False, n=5)

In [ ]:
randomness_func, interval_func = randomness_proportional, interval_proportional
result = randomness_func(df, target_variable)
interval = interval_func(result, step=0.01)

print("Result:", result)
print("Interval:", interval)
random_choice(interval)

In [ ]:
df["randomness"] = df["predicted"].copy()
df["randomness"] = df["predicted"].apply(lambda value: probability_protector(value + random_choice(interval)))

df[features+columns_to_drop+[target_variable,"predicted","randomness"]].sort_values("randomness", ascending=False).head(n=5)

In [ ]:
team_power = 82
opponent_power = 70

y = 1.60e-02 * team_power - 1.45e-02 * opponent_power + 3.78e-01
probability_protector(y+random_choice(interval)) # MODEL IS DONE!

<br><br><br><br>
<h1 style="color:red">Shot on Target</h1>

In [ ]:
features = ['team_power', 'team_finishing', 'opponent_power', 'team_shots']
target_variable = 'team_shots_on_target'

In [ ]:
model, mse, coefficients, intercept = train_linear_regression(df, features, target_variable, int)
mean_error = model_error(df, target_variable).mean()
std_error = model_error(df, target_variable).std()
print("Mean Squared Error:", mse)
print("Error Mean:", mean_error)

In [ ]:
model_equation(coefficients, intercept)

In [ ]:
# plot_predicted_vs_actual(df, target_variable, model=model, features=features)

In [ ]:
model_head(df, features+columns_to_drop, target_variable, sort_parameter="predicted", sort_ascending=False, n=5)

In [ ]:
randomness_func, interval_func = randomness_volume, interval_volume
result = randomness_func(df, target_variable)
interval = interval_func(result, step=1)

print("Result:", result)
print("Interval:", interval)
random_choice(interval)

In [ ]:
df["randomness"] = df["predicted"].copy()
df["randomness"] = df["predicted"].apply(lambda value: negativity_protector(value + random_choice(interval), value))

df[features+columns_to_drop+[target_variable,"predicted","randomness"]].sort_values("randomness", ascending=False).head(n=5)

<br><br><br><br>
<h1 style="color:purple">Shot Models Test</h1>

<br><br><br><br>
<h1 style="color:black">Lineup Test</h1>

In [ ]:
lineup_test_ids = [230621, 235212, 155862, 207865, 252145, 199556, 234153, 230767, 231747, 158023, 190871]
test_team_df = []
for player_id in lineup_test_ids:
    player_df = players_df[players_df['player_id'] == player_id]
    test_team_df.append(player_df)
result_df = pd.concat(test_team_df, ignore_index=True)

In [ ]:
TFR(result_df)["data"]

<br><br><br><br>
<h1 style="color:yellow">"passing"</h1>

In [ ]:
current_stat = "passing"

In [ ]:
cols = FB_MatchLogs(test_stat_df(current_stat))["for"]
ColumnConverter(cols, "For")

In [ ]:
column_names = ['Date', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
                'Opponent', 'Total.Cmp', 'Total.Att', 'Total.Cmp%', 'Total.TotDist',
                'Total.PrgDist', 'Short.Cmp', 'Short.Att', 'Short.Cmp%', 'Medium.Cmp',
                'Medium.Att', 'Medium.Cmp%', 'Long.Cmp', 'Long.Att', 'Long.Cmp%',
                'Unnamed: 23_level_0.Ast', 'Unnamed: 24_level_0.xAG',
                'Unnamed: 25_level_0.xA', 'Unnamed: 26_level_0.KP',
                'Unnamed: 27_level_0.1/3', 'Unnamed: 28_level_0.PPA',
                'Unnamed: 29_level_0.CrsPA', 'Unnamed: 30_level_0.PrgP',
                'Unnamed: 31_level_0.Match Report']

proper_terms = ['Date', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
                'Opponent', 'Total_Completion', 'Total_Attempts', 'Total_Completion_Rate',
                'Total_Total_Distance', 'Total_Progressed_Distance', 'Short_Completion',
                'Short_Attempts', 'Short_Completion_Rate', 'Medium_Completion',
                'Medium_Attempts', 'Medium_Completion_Rate', 'Long_Completion',
                'Long_Attempts', 'Long_Completion_Rate', 'Assists', 'Expected_Goals_Against',
                'Expected_Assists', 'Key_Passes', 'Final_Third_Passes', 'Passes_Progressive',
                'Passes_Into_Final_Third', 'Crosses_And_Corners_Completion', 'Progressive_Passes',
                'Match_Report']

proper_terms = [term.lower().replace(" ", "_") for term in proper_terms]
column_dict = dict(zip(proper_terms, column_names))

prepared_index_dict = column_dict
prepared_rating_data = [tpr_data, tfr_data]

In [ ]:
# df_log_array = generate_fbref_match_logs_array \
# (match_logs_stats_dict, prepared_index_dict, tpr_data, \
#  prepared_rating_data, league_id, team_number_league, last_matchweek, current_stat)
# df_log = pd.DataFrame(df_log_array)
# df_log.to_csv(f"../@blacksmith/match_logs/fbref_match_logs_{league_name}@{current_stat}.csv")

df_log = pd.read_csv(f"../@blacksmith/match_logs/fbref_match_logs_{league_name}@{current_stat}.csv")

In [ ]:
df_log.columns

<br><br><br><br>
<h1 style="color:red">Pass Total Attempts</h1>

In [ ]:
df = df_log
features = ['team_power', 'opponent_power', 'team_total_attempts']
target_variable = 'team_total_completion'

In [ ]:
model, mse, coefficients, intercept = train_linear_regression(df, features, target_variable, int)
mean_error = model_error(df, target_variable).mean()
std_error = model_error(df, target_variable).std()
print("Mean Squared Error:", mse)
print("Error Mean:", mean_error)

In [ ]:
# plot_predicted_vs_actual(df, target_variable, model=model, features=features)

In [ ]:
model_equation(coefficients, intercept)

In [ ]:
model_head(df, features+columns_to_drop, target_variable, sort_parameter="predicted", sort_ascending=True, n=5)

In [ ]:
randomness_func, interval_func = randomness_volume, interval_volume
result = randomness_func(df, target_variable)
interval = interval_func(result, step=15, expand_multiplier=3)

print("Result:", result)
print("Interval:", interval)
random_choice(interval)

In [ ]:
df["randomness"] = df["predicted"].copy()
df["randomness"] = df["predicted"].apply(lambda value: negativity_protector(value + random_choice(interval), value))
df[features+columns_to_drop+[target_variable,"predicted","randomness"]].sort_values("randomness", ascending=True).head(n=5)